In [5]:
#! /usr/bin/env python
import sys
import json
import os
import numpy as np
import pandas as pd

In [6]:
df_sw   = pd.read_json('results/switch.json', orient='records', typ='frame')
df_iotp = pd.read_json('results/iotp.json',   orient='records', typ='frame')
df = pd.concat([df_iotp, df_sw], sort=True)
df[:5]

,aggregation_strategy,avg_delay,data_per_pkt,goodput,gt,pkt_loss,pkt_recv,pkt_send,pps_recv,pps_send,pps_tcpreplay,sample_int,throughput,topology
0,IOTP DT 40,41.419519,1,7.28125,92.753623,0.001073,233,9324,23.3,932.4,1000,10,7.850098,wc
1,IOTP DT 40,41.848961,1,7.18750,92.753623,0.005423,230,9220,23.0,922.0,1000,10,7.749023,wc
2,IOTP DT 40,41.628522,1,7.25000,92.753623,0.006449,232,9304,23.2,930.4,1000,10,7.816406,wc
3,IOTP DT 40,41.914188,1,7.15625,92.753623,0.004903,229,9178,22.9,917.8,1000,10,7.715332,wc
4,IOTP DT 40,41.838511,1,7.21875,92.753623,0.000271,231,9241,23.1,924.1,1000,10,7.782715,wc


In [7]:
def modify_df(df, filename, field, field_data):
    df = pd.concat([df, pd.DataFrame({
        field: [field_data]*len(df)
    })], axis=1)
    df.to_json(filename,   orient='records')
    return df
# modify_df(df_iotp, 'results/iotp.json', 'topology', 'wc')[:10]
# df_sw = df_sw[(df_sw['pps_tcpreplay'] < 6000)]
# df_sw.to_json('results/switch.json',   orient='records')

In [8]:
def show_tests(df):        
    n_l        = []
    strategy_l = []
    dpp_l      = []    
    topo_l     = []
    pps_l      = []
    tests = {
        'N': n_l,
        'Strategy': strategy_l,
        'DPP': dpp_l,
        'Topology': topo_l,
        'PPS': pps_l,
    }        
    column_order = ['N', 'Strategy', 'PPS', 'DPP', 'Topology']
    for pps in df['pps_tcpreplay'].unique():        
        for topo in df['topology'].unique():
            for dpp in df['data_per_pkt'].unique():
                for strategy in df['aggregation_strategy'].unique():                
                    length = len(df[(df['aggregation_strategy'] == strategy) & (df['pps_tcpreplay'] == pps) & (df['data_per_pkt'] == dpp) & (df['topology'] == topo)])
                    n_l.append(length)        
                    strategy_l.append(strategy)
                    dpp_l.append(dpp)
                    topo_l.append(topo)
                    pps_l.append(pps)
    tests_df = pd.DataFrame(tests).sort_values(column_order, ascending=True)
    return tests_df[column_order]

tests = show_tests(df)
for t in tests['Topology'].unique():
    for d in tests['DPP'].unique():
        print "#########  TOPOLOGY %s  -  DPP %d  ##########" %(t,d)        
        print tests[(tests['Topology'] == t) & (tests['DPP'] == d)][['N','Strategy', 'PPS']]
        print " "

#########  TOPOLOGY wc  -  DPP 1  ##########
     N    Strategy   PPS
1   20  IOTP DT 10  1000
55  20  IOTP DT 10  2000
19  20  IOTP DT 10  3000
73  20  IOTP DT 10  4000
37  20  IOTP DT 10  5000
2   20  IOTP DT 20  1000
56  20  IOTP DT 20  2000
20  20  IOTP DT 20  3000
74  20  IOTP DT 20  4000
38  20  IOTP DT 20  5000
3   20  IOTP DT 30  1000
57  20  IOTP DT 30  2000
21  20  IOTP DT 30  3000
75  20  IOTP DT 30  4000
39  20  IOTP DT 30  5000
0   20  IOTP DT 40  1000
54  20  IOTP DT 40  2000
18  20  IOTP DT 40  3000
72  20  IOTP DT 40  4000
36  20  IOTP DT 40  5000
4   20  IOTP DT 50  1000
58  20  IOTP DT 50  2000
22  20  IOTP DT 50  3000
76  20  IOTP DT 50  4000
40  20  IOTP DT 50  5000
5   20   L2 Switch  1000
59  20   L2 Switch  2000
23  20   L2 Switch  3000
77  20   L2 Switch  4000
41  20   L2 Switch  5000
 
#########  TOPOLOGY wc  -  DPP 10  ##########
     N    Strategy   PPS
7   20  IOTP DT 10  1000
61  20  IOTP DT 10  2000
25  20  IOTP DT 10  3000
79  20  IOTP DT 10  4000
43  20 